### Ok this is my fourth attempt.
###### This time I will split each match into two rows, one for the winner, one for the loser. 
###### Different POVs, and an indicator of the victor

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

atp_2022_frame = pd.read_csv('C:/Users/aaron/Code/Data/atp_matches_2022.csv', parse_dates=[5])

atp_2023_frame = pd.read_csv('C:/Users/aaron/Code/Data/atp_matches_2023.csv', parse_dates=[5])

In [2]:
atp_frame = pd.concat([atp_2022_frame, atp_2023_frame], axis = 0)

In [3]:
atp_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5903 entries, 0 to 2985
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tourney_id          5903 non-null   object        
 1   tourney_name        5903 non-null   object        
 2   surface             5850 non-null   object        
 3   draw_size           5903 non-null   int64         
 4   tourney_level       5903 non-null   object        
 5   tourney_date        5903 non-null   datetime64[ns]
 6   match_num           5903 non-null   int64         
 7   winner_id           5903 non-null   int64         
 8   winner_seed         2470 non-null   float64       
 9   winner_entry        901 non-null    object        
 10  winner_name         5903 non-null   object        
 11  winner_hand         5903 non-null   object        
 12  winner_ht           5630 non-null   float64       
 13  winner_ioc          5903 non-null   object      

In [4]:
#### drop rows containing certain null values
atp_frame = atp_frame.dropna(subset = ['winner_age', 'loser_age','winner_rank','loser_rank', 'winner_hand', 'loser_hand', 'winner_ht', 'loser_ht'])

In [5]:
atp_frame.iloc[55:58]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
58,2022-8998,Adelaide 1,Hard,32,A,2022-01-03,296,106423,NaN,WC,Thanasi Kokkinakis,R,196.0,AUS,25.7,144707,NaN,NaN,Mikael Ymer,R,183.0,SWE,23.3,7-6(9) 4-6 6-4,3,QF,164.0,15.0,1.0,103.0,74.0,58.0,11.0,16.0,1.0,3.0,4.0,3.0,106.0,75.0,53.0,17.0,16.0,1.0,3.0,171.0,394.0,94.0,782.0
59,2022-8998,Adelaide 1,Hard,32,A,2022-01-03,295,105227,3.0,NaN,Marin Cilic,R,198.0,CRO,33.2,111513,7.0,NaN,Laslo Djere,R,188.0,SRB,26.5,6-3 6-2,3,QF,67.0,7.0,1.0,43.0,29.0,25.0,11.0,9.0,0.0,0.0,5.0,0.0,47.0,28.0,15.0,11.0,8.0,4.0,7.0,30.0,1710.0,52.0,1131.0
60,2022-8998,Adelaide 1,Hard,32,A,2022-01-03,294,111575,2.0,NaN,Karen Khachanov,R,198.0,RUS,25.6,106078,NaN,Q,Egor Gerasimov,R,196.0,BLR,29.1,7-5 6-3,3,QF,115.0,8.0,1.0,78.0,55.0,40.0,6.0,10.0,5.0,8.0,2.0,6.0,83.0,48.0,32.0,11.0,11.0,7.0,13.0,29.0,1731.0,113.0,665.0


In [6]:

atp_frame['surface_cat'] = atp_frame['surface'].astype("category").cat.codes ## Add a surface category column

atp_frame['winner_hand'] = atp_frame['winner_hand'].astype("category").cat.codes ## convert hand to an int
atp_frame['loser_hand'] = atp_frame['loser_hand'].astype("category").cat.codes ## convert hand to an int    


In [7]:
atp_frame.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'surface_cat'],
      dtype='object')

In [8]:
### make df for just winners
#columns_to_keep_for_winners = ['tourney_name','surface','match_num','winner_id','winner_name','winner_hand','winner_ht','winner_age', 'minutes',
 #                              'w_ace', 'w_df','w_svpt','w_1stIn','w_1stWon','w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'winner_rank']




winning_rename_lib = {'winner_id':'player_id', 'winner_seed':'player_seed', 'winner_entry':'player_entry',
       'winner_name':'player_name', 'winner_hand':'player_hand', 'winner_ht':'player_ht', 'winner_ioc':'player_ioc', 'winner_age':'player_age',
       'loser_id':'opponent_id', 'loser_seed':'opponent_seed', 'loser_entry':'opponent_entry', 'loser_name':'opponent_name', 'loser_hand':'opponent_hand',
       'loser_ht': 'opponent_ht', 'loser_ioc':'opponent_ioc', 'loser_age':'opponent_age',
        'w_ace':'player_ace', 'w_df':'player_df', 'w_svpt':'player_svpt', 'w_1stIn':'player_1stIn', 'w_1stWon':'player_1stWon', 'w_2ndWon':'player_2ndWon',
       'w_SvGms':'player_SvGms', 'w_bpSaved':'player_bpSaved', 'w_bpFaced':'player_bpFaced', 'l_ace':'opponent_ace', 'l_df':'opponent_df', 'l_svpt':'opponent_svpt',
       'l_1stIn':'opponent_1stIn', 'l_1stWon':'opponent_1stWon', 'l_2ndWon':'opponent_2ndWon', 'l_SvGms':'opponent_SvGms', 'l_bpSaved':'opponent_bpSaved', 'l_bpFaced':'opponent_bpFaced',
       'winner_rank':'player_rank', 'winner_rank_points':'player_rank_points', 'loser_rank':'opponent_rank', 'loser_rank_points':'opponent_rank_points'}

losing_rename_lib = {'winner_id':'opponent_id', 'winner_seed':'opponent_seed', 'winner_entry':'opponent_entry',
       'winner_name':'opponent_name', 'winner_hand':'opponent_hand', 'winner_ht':'opponent_ht', 'winner_ioc':'opponent_ioc', 'winner_age':'opponent_age',
       'loser_id':'player_id', 'loser_seed':'player_seed', 'loser_entry':'player_entry', 'loser_name':'player_name', 'loser_hand':'player_hand',
       'loser_ht': 'player_ht', 'loser_ioc':'player_ioc', 'loser_age':'player_age',
        'w_ace':'opponent_ace', 'w_df':'opponent_df', 'w_svpt':'opponent_svpt', 'w_1stIn':'opponent_1stIn', 'w_1stWon':'opponent_1stWon', 'w_2ndWon':'opponent_2ndWon',
       'w_SvGms':'opponent_SvGms', 'w_bpSaved':'opponent_bpSaved', 'w_bpFaced':'opponent_bpFaced', 'l_ace':'player_ace', 'l_df':'player_df', 'l_svpt':'player_svpt',
       'l_1stIn':'player_1stIn', 'l_1stWon':'player_1stWon', 'l_2ndWon':'player_2ndWon', 'l_SvGms':'player_SvGms', 'l_bpSaved':'player_bpSaved', 'l_bpFaced':'player_bpFaced',
       'winner_rank':'opponent_rank', 'winner_rank_points':'opponent_rank_points', 'loser_rank':'player_rank', 'loser_rank_points':'player_rank_points'}

#losing_rename_lib = {'loser_rank':'player_rank', 'winner_rank':'opponent_rank', 'loser_age':'player_age', 'winner_age':'opponent_age', 'loser_ht':'player_height', 'winner_ht':'opponent_height', 'loser_hand':'player_hand', 'winner_hand':'opponent_hand'} #rename columns
    
winners_df = atp_frame.rename(columns = winning_rename_lib)
losers_df = atp_frame.rename(columns = losing_rename_lib)

#winners_of_2022_df = atp_2022_frame[columns_to_keep_for_winners]

losers_df['result'] = 0
winners_df['result'] = 1

each_match_twice = pd.concat([winners_df, losers_df], axis = 0)

#each_match_twice_2022 = winners_of_2022_df.append(losers_of_2022_df)#pd.concat([winners_of_2022_df, losers_of_2022_df])

In [9]:
 ### trying to make sure each player's matches are in consecutive order (i think im satisfied with the result)
each_match_twice = each_match_twice.sort_values(by = ['tourney_date', 'tourney_id','match_num'])

In [10]:
#### make some columns for things like break point coverted percent, ace to df percent, bp saved percent .... etcetera

each_match_twice['player_break_pct'] = (each_match_twice['opponent_bpFaced'] - each_match_twice['opponent_bpSaved']) / each_match_twice['opponent_SvGms']
each_match_twice['player_hold_pct'] = (each_match_twice['player_SvGms'] - (each_match_twice['player_bpFaced'] - each_match_twice['player_bpSaved'])) / each_match_twice['player_SvGms']

In [11]:
each_match_twice.iloc[350:359]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,score,best_of,round,minutes,player_ace,player_df,player_svpt,player_1stIn,player_1stWon,player_2ndWon,player_SvGms,player_bpSaved,player_bpFaced,opponent_ace,opponent_df,opponent_svpt,opponent_1stIn,opponent_1stWon,opponent_2ndWon,opponent_SvGms,opponent_bpSaved,opponent_bpFaced,player_rank,player_rank_points,opponent_rank,opponent_rank_points,surface_cat,result,player_break_pct,player_hold_pct
179,2022-580,Australian Open,Hard,128,G,2022-01-17,117,105357,NaN,NaN,John Millman,1,183.0,AUS,32.5,103852,NaN,NaN,Feliciano Lopez,0,188.0,ESP,40.3,6-1 6-3 4-6 7-5,5,R128,172.0,6.0,0.0,121.0,77.0,55.0,25.0,19.0,5.0,7.0,26.0,5.0,132.0,78.0,57.0,18.0,19.0,15.0,21.0,89.0,788.0,109.0,696.0,2,1,0.315789,0.894737
179,2022-580,Australian Open,Hard,128,G,2022-01-17,117,103852,NaN,NaN,Feliciano Lopez,0,188.0,ESP,40.3,105357,NaN,NaN,John Millman,1,183.0,AUS,32.5,6-1 6-3 4-6 7-5,5,R128,172.0,26.0,5.0,132.0,78.0,57.0,18.0,19.0,15.0,21.0,6.0,0.0,121.0,77.0,55.0,25.0,19.0,5.0,7.0,109.0,696.0,89.0,788.0,2,0,0.105263,0.684211
180,2022-580,Australian Open,Hard,128,G,2022-01-17,118,105430,NaN,Q,Radu Albot,1,175.0,MDA,32.1,106415,NaN,NaN,Yoshihito Nishioka,0,170.0,JPN,26.3,6-3 6-4 4-6 6-2,5,R128,154.0,7.0,5.0,103.0,66.0,52.0,20.0,19.0,1.0,4.0,3.0,2.0,109.0,66.0,42.0,23.0,18.0,2.0,8.0,124.0,587.0,119.0,610.0,2,1,0.333333,0.842105
180,2022-580,Australian Open,Hard,128,G,2022-01-17,118,106415,NaN,NaN,Yoshihito Nishioka,0,170.0,JPN,26.3,105430,NaN,Q,Radu Albot,1,175.0,MDA,32.1,6-3 6-4 4-6 6-2,5,R128,154.0,3.0,2.0,109.0,66.0,42.0,23.0,18.0,2.0,8.0,7.0,5.0,103.0,66.0,52.0,20.0,19.0,1.0,4.0,119.0,610.0,124.0,587.0,2,0,0.157895,0.666667
181,2022-580,Australian Open,Hard,128,G,2022-01-17,119,126846,NaN,WC,Aleksandar Vukic,1,188.0,AUS,25.7,144750,30.0,NaN,Lloyd Harris,1,193.0,RSA,24.8,4-6 6-3 7-5 7-6(3),5,R128,175.0,11.0,1.0,142.0,82.0,68.0,36.0,22.0,4.0,5.0,14.0,2.0,134.0,95.0,70.0,21.0,21.0,7.0,9.0,144.0,477.0,33.0,1473.0,2,1,0.095238,0.954545
181,2022-580,Australian Open,Hard,128,G,2022-01-17,119,144750,30.0,NaN,Lloyd Harris,1,193.0,RSA,24.8,126846,NaN,WC,Aleksandar Vukic,1,188.0,AUS,25.7,4-6 6-3 7-5 7-6(3),5,R128,175.0,14.0,2.0,134.0,95.0,70.0,21.0,21.0,7.0,9.0,11.0,1.0,142.0,82.0,68.0,36.0,22.0,4.0,5.0,33.0,1473.0,144.0,477.0,2,0,0.045455,0.904762
182,2022-580,Australian Open,Hard,128,G,2022-01-17,120,124187,23.0,NaN,Reilly Opelka,1,211.0,USA,24.3,104731,NaN,NaN,Kevin Anderson,1,203.0,RSA,35.6,6-3 6-4 7-6(3),5,R128,104.0,23.0,0.0,76.0,56.0,51.0,18.0,16.0,0.0,0.0,11.0,3.0,83.0,53.0,44.0,16.0,15.0,1.0,3.0,29.0,1776.0,97.0,744.0,2,1,0.133333,1.000000
182,2022-580,Australian Open,Hard,128,G,2022-01-17,120,104731,NaN,NaN,Kevin Anderson,1,203.0,RSA,35.6,124187,23.0,NaN,Reilly Opelka,1,211.0,USA,24.3,6-3 6-4 7-6(3),5,R128,104.0,11.0,3.0,83.0,53.0,44.0,16.0,15.0,1.0,3.0,23.0,0.0,76.0,56.0,51.0,18.0,16.0,0.0,0.0,97.0,744.0,29.0,1776.0,2,0,0.000000,0.866667
183,2022-580,Australian Open,Hard,128,G,2022-01-17,121,136440,NaN,NaN,Dominik Koepfer,0,180.0,GER,27.7,126535,NaN,NaN,Carlos Taberner,1,183.0,ESP,24.4,6-1 3-6 6-4 6-1,5,R128,147.0,7.0,5.0,88.0,60.0,47.0,14.0,17.0,2.0,4.0,3.0,6.0,101.0,51.0,35.0,19.0,16.0,3.0,10.0,53.0,1096.0,108.0,707.0,2,1,0.437500,0.882353


In [12]:
grouped_matches = each_match_twice.groupby('player_name')

In [13]:
single_player = grouped_matches.get_group('John Millman')

#single_player

In [14]:
def rolling_averages(group, columns, new_columns):
    rolling_stats = group[columns].rolling(5, closed='left').mean()
    group[new_columns] = rolling_stats
    group = group.dropna(subset=new_columns)
    return group

In [15]:
columns = ['player_df', 'player_1stIn','player_1stWon', 'player_2ndWon', 'result', 'player_break_pct', 'player_hold_pct']
new_columns = [f"{c}_rolling" for c in columns]

In [16]:
rolling_averages(single_player, columns, new_columns)

C:\Users\aaron\AppData\Local\Temp\ipykernel_20488\713556541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group[new_columns] = rolling_stats
C:\Users\aaron\AppData\Local\Temp\ipykernel_20488\713556541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group[new_columns] = rolling_stats
C:\Users\aaron\AppData\Local\Temp\ipykernel_20488\713556541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,score,best_of,round,minutes,player_ace,player_df,player_svpt,player_1stIn,player_1stWon,player_2ndWon,player_SvGms,player_bpSaved,player_bpFaced,opponent_ace,opponent_df,opponent_svpt,opponent_1stIn,opponent_1stWon,opponent_2ndWon,opponent_SvGms,opponent_bpSaved,opponent_bpFaced,player_rank,player_rank_points,opponent_rank,opponent_rank_points,surface_cat,result,player_break_pct,player_hold_pct,player_df_rolling,player_1stIn_rolling,player_1stWon_rolling,player_2ndWon_rolling,result_rolling,player_break_pct_rolling,player_hold_pct_rolling
476,2022-0499,Delray Beach,Hard,32,A,2022-02-14,277,105357,NaN,NaN,John Millman,1,183.0,AUS,32.6,202195,8.0,NaN,Maxime Cressy,1,198.0,USA,24.7,6-7(2) 7-6(2) 7-6(3),3,R32,173.0,5.0,5.0,121.0,72.0,57.0,32.0,18.0,4.0,5.0,12.0,8.0,111.0,66.0,54.0,29.0,18.0,4.0,5.0,91.0,738.0,59.0,1047.0,2,1,0.055556,0.944444,0.8,47.0,29.8,15.8,0.2,0.095209,0.693306
466,2022-0499,Delray Beach,Hard,32,A,2022-02-14,289,105357,NaN,NaN,John Millman,1,183.0,AUS,32.6,106218,NaN,NaN,Marcos Giron,1,180.0,USA,28.5,6-3 7-6(6),3,R16,90.0,5.0,1.0,65.0,46.0,38.0,11.0,11.0,0.0,1.0,5.0,0.0,55.0,37.0,28.0,11.0,10.0,0.0,2.0,91.0,738.0,64.0,988.0,2,1,0.200000,0.909091,1.8,53.8,36.2,20.2,0.4,0.106320,0.742195
460,2022-0499,Delray Beach,Hard,32,A,2022-02-14,295,105357,NaN,NaN,John Millman,1,183.0,AUS,32.6,105777,3.0,NaN,Grigor Dimitrov,1,188.0,BUL,30.7,6-4 6-4,3,QF,94.0,1.0,1.0,59.0,38.0,27.0,14.0,10.0,2.0,3.0,4.0,1.0,61.0,33.0,21.0,14.0,10.0,3.0,6.0,91.0,738.0,26.0,1786.0,2,1,0.300000,0.900000,1.4,55.6,39.2,21.2,0.6,0.146320,0.824013
457,2022-0499,Delray Beach,Hard,32,A,2022-02-14,298,105357,NaN,NaN,John Millman,1,183.0,AUS,32.6,124187,2.0,NaN,Reilly Opelka,1,211.0,USA,24.4,7-6(8) 6-7(3) 7-6(4),3,SF,180.0,3.0,2.0,137.0,93.0,65.0,29.0,18.0,9.0,10.0,16.0,2.0,109.0,73.0,60.0,27.0,18.0,1.0,2.0,91.0,738.0,20.0,2051.0,2,0,0.055556,0.944444,1.6,47.8,33.6,19.0,0.6,0.143162,0.825066
587,2022-0807,Acapulco,Hard,32,A,2022-02-21,276,105357,NaN,SE,John Millman,1,183.0,AUS,32.6,106218,NaN,NaN,Marcos Giron,1,180.0,USA,28.5,7-6(3) 2-0 RET,3,R32,93.0,3.0,0.0,58.0,36.0,22.0,9.0,7.0,6.0,9.0,3.0,2.0,58.0,32.0,23.0,11.0,7.0,3.0,5.0,80.0,818.0,59.0,990.0,2,0,0.285714,0.571429,2.0,58.6,43.0,20.6,0.6,0.138889,0.906263
737,2022-0404,Indian Wells Masters,Hard,128,M,2022-03-07,192,105357,NaN,LL,John Millman,1,183.0,AUS,32.7,105526,NaN,NaN,Jan Lennard Struff,1,193.0,GER,31.8,7-6(6) 6-4,3,R128,93.0,5.0,2.0,72.0,49.0,37.0,13.0,11.0,5.0,6.0,16.0,3.0,79.0,46.0,34.0,15.0,11.0,6.0,8.0,77.0,818.0,58.0,984.0,2,1,0.181818,0.909091,1.8,57.0,41.8,19.0,0.6,0.179365,0.853882
705,2022-0404,Indian Wells Masters,Hard,128,M,2022-03-07,247,105357,NaN,LL,John Millman,1,183.0,AUS,32.7,200282,29.0,NaN,Alex De Minaur,1,183.0,AUS,23.0,7-6(4) 6-3,3,R64,100.0,1.0,0.0,63.0,35.0,21.0,14.0,10.0,1.0,4.0,5.0,0.0,65.0,38.0,26.0,18.0,11.0,0.0,2.0,77.0,818.0,31.0,1451.0,2,0,0.181818,0.700000,1.2,52.4,37.8,15.2,0.6,0.204618,0.846811
835,2022-0403,Miami Masters,Hard,128,M,2022-03-21,187,105357,NaN,NaN,John Millman,1,183.0,AUS,32.7,123755,NaN,Q,Daniel Elahi Galan,1,191.0,COL,25.7,6-4 6-2,3,R128,82.0,2.0,0.0,57.0,34.0,18.0,10.0,9.0,4.0,9.0,3.0,0.0,51.0,32.0,19.0,13.0,9.0,3.0,5.0,73.0,838.0,118.0,554.0,2,0,0.222222,0.444444,1.0,50.2,34.4,15.8,0.4,0.200981,0.804993
1018,2022-5053,Belgrade,Clay,32,A,2022-04-18,283,105357,NaN,NaN,John Millman,1,183.0,AUS,32.8,106233,NaN,NaN,Dominic Thiem,1,185.0,AUT,28.6,6-3 3-6 6-4,3,R32,153.0,3.0,1.0,92.0,55.0,32.0,27.0,14.0,4.0,6.0,4.0,2.0,93.0,56.0,35.0,21.0,14.0,3.0,6.0,80.0,793.0,54.0,975.0,0,1,0.214286,0.857143,0.8,49.4,32.6,15.0,0.2,0.185426,0.713882
1010,2022-5053,Belgrade,Clay,32,A,2022-04-18,292,105357,NaN,NaN,John Millman,1,183.0,AUS,32.8,200175,7.0,NaN,Miomir Kecmanovic,1,183.0,SRB

In [17]:
## apply the #rolling function to each 'player' group in the dataframe
matches_rolling = each_match_twice.groupby("player_name").apply(lambda x: rolling_averages(x, columns, new_columns))

In [18]:
## This is kinda like reset_index(), it gets rid of the left column
matches_rolling = matches_rolling.droplevel('player_name')

In [19]:
### adding an index by shape
matches_rolling.index = range(matches_rolling.shape[0])
#matches_rolling_2023.index = range(matches_rolling_2023.shape[0])

In [20]:
random_forest = RandomForestClassifier(n_estimators=50, min_samples_split=20, random_state=1, verbose = 1)

In [21]:
predictors = ["player_rank", "opponent_rank", "player_age", "opponent_age",
              "player_ht", "opponent_ht", "surface_cat", "player_hand", "opponent_hand", 
              "player_df_rolling", "player_1stIn_rolling", "player_1stWon_rolling", "player_2ndWon_rolling", "result_rolling", 
              "player_break_pct_rolling", "player_hold_pct_rolling"]

simple_preds = ["player_age", "opponent_age",
                "player_ht", "opponent_ht", "player_hand", "opponent_hand", "surface_cat",
               "player_df_rolling", "player_1stIn_rolling", "player_1stWon_rolling", "player_2ndWon_rolling"]

super_simple_preds = ["player_rank", "opponent_rank"]

In [22]:
#### after combining earlier, lets now split into 2022 and 2023 data.
matches_rolling_2022 = matches_rolling[matches_rolling['tourney_date'] < '2022-12-30']
matches_rolling_2023 = matches_rolling[matches_rolling['tourney_date'] > '2023-1-01']

In [23]:
random_forest.fit(matches_rolling_2022[predictors], matches_rolling_2022['result'])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    1.7s


RandomForestClassifier(min_samples_split=20, n_estimators=50, random_state=1,
                       verbose=1)

In [24]:
predictions = random_forest.predict(matches_rolling_2023[predictors])

predictions.shape

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


(4198,)

In [25]:
accuracy = accuracy_score(matches_rolling_2023['result'], predictions)

accuracy

0.6283944735588376